In [1]:
from utils.extract_pdf_contents import process_pdfs
from utils.clean_texts import clean_EU_legal_text, clean_US_legal_text
from utils.TFIDF_model import load_text, compute_tfidf, save_tfidf_values
from sklearn.feature_extraction.text import CountVectorizer
import spacy
import nltk
import spacy


## Research paper NLP Pipeline

### 1. Extract the text from the two pdf documents

In [2]:
# Define the input and output directories
input_directory = "data/raw"  # Directory containing the PDF files
output_directory = "data/extracted_text"  # Directory to save the extracted text files


# Call the function to process the PDFs
process_pdfs(input_directory, output_directory)

Extracted text saved to: data/extracted_text/EU_AI_Act_English.txt
Extracted text saved to: data/extracted_text/USA_AI_Executive_Order_English.txt


### 2. Cleanup the extracted text

In [3]:
# First the EU Act

# Read the file content and pass it to clean_legal_text
with open("data/extracted_text/EU_AI_Act_English.txt", "r", encoding="utf-8") as file:
    text = file.read()

clean_EU_text = clean_EU_legal_text(text)

# Save cleaned text to /data/cleaned_text directory
with open("data/cleaned_text/EU_AI_Act_English_Cleaned.txt", "w", encoding="utf-8") as file:
    file.write(clean_EU_text)


# Second, the USA AI Executive Order

# Read the file content and pass it to clean_legal_text
with open("data/extracted_text/USA_AI_Executive_Order_English.txt", "r", encoding="utf-8") as file:
    text = file.read()
    clean_US_text = clean_US_legal_text(text)

# Save cleaned text to /data/cleaned_text directory
with open("data/cleaned_text/USA_AI_Executive_Order_English_Cleaned.txt", "w", encoding="utf-8") as file:
    file.write(clean_US_text)

### 3. Tokenization

In [3]:
# Paths to the cleaned text files
text_file_1 = "data/cleaned_text/EU_AI_Act_English_Cleaned.txt"
text_file_2 = "data/cleaned_text/USA_AI_Executive_Order_English_Cleaned.txt"

# Load the cleaned texts
text1 = load_text(text_file_1)
text2 = load_text(text_file_2)


# JUSTIFY THE CHOICES
spacy.cli.download("en_core_web_sm")
nlp = spacy.load("en_core_web_sm", disable=["tok2vec", "tagger", "parser", "ner",
                                                "lemmatizer", "attibute_ruler"]) 
nltk.download("stopwords")
stop_words_en = nltk.corpus.stopwords.words("english")

# Custom tokenization function
def custom_tokenizer(text): # wrap tokenizer in custom function
    tokenized_text = nlp(text)
    # Remove empty tokens
    return [tok.text.strip() for tok in tokenized_text if tok.text.strip() !='']

# Manual tokenization using the custom tokenizer (for inspection purposes)
tokens_EU_AI = custom_tokenizer(text1)
tokens_US_AI = custom_tokenizer(text2)

# Saving the tokens to files for inspection
with open("data/tokens/EU_AI_Act_Tokens.txt", "w", encoding="utf-8") as f:
    for token in tokens_EU_AI:
        f.write(f"{token}\n")

with open("data/tokens/USA_AI_Executive_Order_Tokens.txt", "w", encoding="utf-8") as f:
    for token in tokens_US_AI:
        f.write(f"{token}\n")

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 12.8/12.8 MB 4.9 MB/s  0:00:02 eta 0:00:01
✔ Download and installation successful
You can now load the package via spacy.load('en_core_web_sm')


[nltk_data] Downloading package stopwords to
[nltk_data]     /Users/nicolasreichardt/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


### 4. Feature extraction

#### 4.1. Bag of Words (BoW)

In [4]:
# Vectorizer Configuration for BoW (required for LDA)
tokenizer_bow = CountVectorizer(analyzer="word",
                                       tokenizer=custom_tokenizer,
                                       lowercase=True,
                                       stop_words=stop_words_en)
                                      # max_df= 0.80, # Ignore terms that appear in more than 80% of the documents
                                      # min_df= 0.01) # Ignore terms that appear in less than 1% of the documents
text1_bow = tokenizer_bow.fit_transform([text1])
print(f"BoW Matrix Dimensions EU AI Act: {text1_bow.shape}")

text2_bow = tokenizer_bow.fit_transform([text2])
print(f"BoW Matrix Dimensions USA AI Executive Order: {text2_bow.shape}")

/opt/miniconda3/envs/nlp-research-note/lib/python3.10/site-packages/sklearn/feature_extraction/text.py:525: UserWarning: The parameter 'token_pattern' will not be used since 'tokenizer' is not None'
  warnings.warn(
/opt/miniconda3/envs/nlp-research-note/lib/python3.10/site-packages/sklearn/feature_extraction/text.py:408: UserWarning: Your stop_words may be inconsistent with your preprocessing. Tokenizing the stop words generated tokens ["'d", "'ll", "'m", "'re", "'s", "'ve", 'could', 'might', 'must', "n't", 'need', 'sha', 'wo', 'would'] not in stop_words.
  warnings.warn(


BoW Matrix Dimensions EU AI Act: (1, 4823)
BoW Matrix Dimensions USA AI Executive Order: (1, 2933)


#### 4.2. TF-IDF

In [5]:
if text1 and text2:
    # Compute TF-IDF
    feature_names, tfidf_matrix = compute_tfidf([text1, text2])
    
    # Save TF-IDF tokens
    output_directory = "data/tfidf_values"
    save_tfidf_values(output_directory, feature_names, tfidf_matrix)

TF-IDF values saved to: data/tfidf_values/tfidf_doc_1.txt
TF-IDF values saved to: data/tfidf_values/tfidf_doc_2.txt


#### 4.3. Embeddings (Word2Vec)

In [6]:
# Import the Word2VecModel class
from utils.word2vec_module import Word2VecModel

# Create and train the Word2Vec model
embedding_documents = [tokens_EU_AI, tokens_US_AI]
word2vec = Word2VecModel(vector_size=100, # size of the embedding vectors
                         window=5, # context window size
                         min_count=2, # minimum frequency for a word to be included
                         workers=4) # number of CPU cores to use

word2vec.train(embedding_documents)

# Save the trained model
word2vec.save("data/embeddings/word2vec")

# Example: Get embedding for a specific word
get_vector = word2vec.get_vector("AI")
print(f"Vector for 'AI': {get_vector}")


Vector for 'AI': [-0.86636376  0.548325    0.25130495  0.09696353  0.04953064 -0.75212425
  1.0235596   1.2100562  -0.71805704 -0.57815415 -0.10712975 -0.78561515
  0.35065737  0.36541373 -0.06964274 -0.00711733  0.4334405  -0.66818035
 -0.60364914 -1.549883    0.7385225   0.2622882   0.9610372  -0.66926694
 -0.04522961  0.29183722 -0.5660738   0.312025   -0.87127185  0.7419106
  0.939304    0.1480871   0.15538937 -1.2829772   0.09045626  0.68759465
  0.22452477 -0.56933266 -0.8324655  -0.3630775  -0.08976114 -0.721901
 -0.13080858  0.45045444  1.2714486  -0.2353045  -0.5306903   0.18124744
 -0.00663342  0.33188397  0.74116325 -0.19368352 -0.45823205 -0.08577094
 -0.9203496  -0.14333576  0.6694571  -0.35781494 -0.6405909   0.3104716
 -0.41222313 -0.4981957   0.6006506  -0.03400317 -0.7551183   0.67135555
  0.30413777  0.9910026  -1.1352624   0.66105336 -0.39192253  0.7527838
  0.80577666  0.21845265  0.4602903  -0.08216729  0.3377547   0.15612938
 -0.20970806  0.3504927  -0.84434855  0

#### 4.4. (Optional) Text representation through OpenAI contextualised embeddings

### 5. Mathematical analysis

#### 5.1. Similarity Measurement (Cosine Similiarity)

In [7]:
# WORK IN PROGRESS - Error

# Example: Find similar words
similar_words = word2vec.most_similar("AI", topn=5)

print(f"Words similar to 'AI': {similar_words}")

# Example: Compute similarity between two words
similarity = word2vec.similarity("AI", "artificial")

print(f"Similarity between 'AI' and 'artificial': {similarity}")

# Example: Compute similarity between two documents
doc1_vector = word2vec.document_vector(tokens_EU_AI)
doc2_vector = word2vec.document_vector(tokens_US_AI)
doc_similarity = word2vec.cosine_similarity(doc1_vector, doc2_vector)
print(f"Similarity between EU AI Act and USA AI Executive Order: {doc_similarity}")

# Example: Find most similar words to a document
most_similar_to_doc = word2vec.most_similar_to_document(tokens_EU_AI, topn=5)
print(f"Words most similar to EU AI Act: {most_similar_to_doc}")


AttributeError: 'Word2VecModel' object has no attribute 'most_similar'

#### 5.2. Dimensionality Reduction (t-SNE)

In [8]:
# t-SNE Visualization
from utils.tsne_visualization import plot_tsne_embeddings
import numpy as np

# Prepare embeddings for visualization
words = list(word2vec.model.wv.index_to_key) # Get all words in the vocabulary
embeddings = np.array([word2vec.get_vector(word) for word in words])  # Convert to NumPy array for t-SNE function to work

# Plot t-SNE embeddings and save
plot_tsne_embeddings(embeddings, words, "plots/tsne_embeddings.png")


#### 5.2. Dimensionality Reduction (PCA) - Optional?

#### 5.3. Other techniques?

### 5. Topic modeling

#### 5.1. Latent Dirichlet Allocation (LDA)

#### 5.2. BERTopic